In [1]:
df = sqlContext.sql("select * from mall_customers")
df = df.drop('CustomerID')
df.printSchema()

In [2]:
from pyspark.ml.feature import StandardScaler,VectorAssembler,VectorIndexer,OneHotEncoder,StringIndexer
from pyspark.ml.clustering import KMeans

In [3]:
indexer = StringIndexer(inputCol="Genre", outputCol="Sex")
indexed = indexer.fit(df).transform(df)

In [4]:
assembler = VectorAssembler(inputCols=['Age', 'Annual Income (k$)', 'Spending Score (1-100)', 'Sex'],outputCol="features")
output = assembler.transform(indexed)
final_data = output.select("features")

In [5]:
scaler = StandardScaler(inputCol="features", outputCol="scaledFeatures", withStd=True, withMean=False)
scalerModel = scaler.fit(final_data)
cluster_final_data = scalerModel.transform(final_data)

In [6]:
for k in range(2,11):
    kmeans = KMeans(featuresCol='scaledFeatures',k=k)
    model = kmeans.fit(cluster_final_data)
    wssse = model.computeCost(cluster_final_data)
    print("With K={}".format(k))
    print("Within Set Sum of Squared Errors = " + str(wssse))
    print('--'*30)

In [7]:
kmeans = KMeans(featuresCol='scaledFeatures',k=6)
model_kmeans = kmeans.fit(cluster_final_data)
model_kmeans.transform(cluster_final_data).groupBy('prediction').count().show()
print('\n')
centers = model_kmeans.clusterCenters()
print("Cluster Centers: ")
for center in centers:
    print(center)